In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import skimage.transform
import matplotlib.pyplot as plt

import itertools
from typing import Tuple

In [2]:
tf.test.is_gpu_available()

True

## Load training data

Load data and create shifted versions.  
This is not redundant, even though we are using convolutional layers, as there are boundary effects (especially with the very small size).

In [3]:
WIDTH = 5
HEIGHT = 5

with open("data.txt") as f:
    t = f.read()

xs = []
ys = []

for ex in t.split("\n\n"):
    lines = ex.split("\n")
    y  = int(lines[0])
    x = np.array([[0 if x == "." else 1 for x in line] for line in lines[1:]])
    
    ex_width = np.max(np.arange(1, WIDTH + 1) * x)
    ex_height = np.max((np.arange(1, HEIGHT + 1) * x.T).T)
    print(y, ex_width, ex_height, end="")
    for offset_x, offset_y in itertools.product(range(WIDTH - ex_width + 1),
                                                range(HEIGHT - ex_height + 1)):
        xs.append(np.roll(x, (offset_y, offset_x), axis=(0, 1)))
        ys.append(y)
        print(".", end="")
    print()

xs = np.array(xs).astype(float)[..., np.newaxis]
ys = np.array(ys)

0 3 3.........
0 3 3.........
0 4 4....
0 3 4......
0 3 4......
0 3 4......
0 3 5...
0 4 4....
0 4 5..
0 4 4....
0 4 5..
0 5 5.
1 1 5.....
1 1 4..........
1 1 3...............
1 3 3.........
1 2 4........
1 3 4......
1 3 4......
1 3 5...
1 3 4......
1 3 4......
2 5 5.
2 3 3.........
2 3 5...
2 5 5.
2 4 5..
2 5 5.
2 5 5.
2 5 5.
2 3 4......
2 3 4......
2 3 5...
2 3 4......
3 3 3.........
3 2 3............
3 4 5..
3 4 5..
3 3 5...
3 2 5....
3 4 5..
3 5 5.
3 3 5...
3 5 5.
3 5 5.
3 4 5..
3 3 5...
3 3 5...
3 4 5..
3 4 5..
3 3 3.........
3 3 4......
3 3 4......
3 3 4......
4 4 5..
4 4 5..
4 5 5.
4 4 5..
4 4 5..
4 3 3.........
4 4 4....
4 4 4....
4 5 5.
4 4 5..
4 3 4......
4 3 4......
5 4 5..
5 4 5..
5 3 5...
5 3 5...
5 3 5...
5 4 5..
5 5 5.
5 2 4........
5 3 4......
5 3 3.........
5 3 4......
5 3 4......
5 3 4......
5 4 4....
6 3 5...
6 3 4......
6 3 5...
6 3 3.........
6 3 5...
6 2 4........
6 3 4......
6 3 4......
6 3 4......
6 4 4....
7 3 4......
7 4 5..
7 5 5.
7 5 5.
7 3 4......
7 3 3....

\# of (shifted) examples per class

In [46]:
len(ys)

534

In [4]:
pd.Series(ys).value_counts()

3    79
1    74
5    61
0    56
9    54
6    54
8    44
4    41
2    40
7    31
dtype: int64

In [5]:
i = np.random.randint(0, len(ys))
print(ys[i])
print(xs[i,:,:,0].astype(int))

5
[[0 0 0 0 0]
 [0 0 1 1 0]
 [0 0 1 0 0]
 [0 1 1 0 0]
 [0 0 0 0 0]]


In [6]:
def random_flip(x: np.ndarray, xs_strings: np.ndarray) -> np.ndarray:
    """Randomly flips a bit in x, ensuring that the resulting pattern
       does not occurr in xs_strings
    """
    while True:
        x_new = x.copy()
        pos_y = np.random.randint(0, HEIGHT)
        pos_x = np.random.randint(0, WIDTH)
        x_new[pos_y, pos_x] = 1 - x_new[pos_y, pos_x]
        if x_new.tostring() not in xs_strings:
            return x_new

In [7]:
xs_strings = {x.tostring() for x in xs}

In [52]:
BATCH_SIZE = 512

ds = tf.data.Dataset.zip((
    tf.data.Dataset.from_tensor_slices(xs),
    tf.data.Dataset.from_tensor_slices(ys)
)).shuffle(buffer_size=len(ys)).repeat()

def augment(x: np.ndarray) -> np.ndarray:
    """Flip up to 3 bits randomly"""
    if np.random.uniform() > 0.5:
        x = random_flip(x, xs_strings)
    if np.random.uniform() > 0.85:
        x = random_flip(x, xs_strings)
    if np.random.uniform() > 0.95:
        x = random_flip(x, xs_strings)
    
    return x
    

def map_fn(x: tf.Tensor, y: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = tf.numpy_function(func=augment , inp=[x], Tout=tf.float64)
    x.set_shape([HEIGHT, WIDTH, 1])
    return x, y

ds = ds.map(map_fn, num_parallel_calls=8)
ds = ds.batch(BATCH_SIZE)


In [53]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(4, 2, activation="relu", padding="valid", input_shape=(WIDTH, HEIGHT, 1)),
    tf.keras.layers.Conv2D(8, 2,  activation="relu"),
    tf.keras.layers.Conv2D(16, 2,  activation="relu"),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax)
])

In [54]:
model.count_params()

412

In [55]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025), metrics=["accuracy"], loss=tf.keras.losses.sparse_categorical_crossentropy)

In [ ]:
# training is fast, so let's just go crazy with the number of epochs
for i in range(50):
    model.fit(ds, epochs=100, verbose=0, steps_per_epoch=len(ys) // BATCH_SIZE)
    print(model.evaluate(xs, ys, verbose=0))

In [59]:
model.optimizer.lr.assign(0.0005)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5e-05>

In [ ]:
for i in range(10):
    model.fit(ds, epochs=100, verbose=0, steps_per_epoch=len(ys) // BATCH_SIZE)
    print(model.evaluate(xs, ys, verbose=0))

In [86]:
model.evaluate(ds, steps=len(ys) // BATCH_SIZE)

1/1 [==============================] - 1s 712ms/step - loss: 0.3415 - accuracy: 0.8945


[0.3414769768714905, 0.89453125]

In [87]:
print(model.evaluate(xs, ys, verbose=0))

[0.06978682071417013, 1.0]


In [17]:
model.save("5x5_4-8-16_filters.savedmodel")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: 5x5_4-8-16_filters.savedmodel\assets


In [85]:
model = tf.keras.models.load_model("5x5_4-8-16_filters.savedmodel/")

In [62]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 4, 4, 4)           20        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 3, 8)           136       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 16)          528       
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                170       
Total params: 854
Trainable params: 854
Non-trainable params: 0
_________________________________________________________________


In [66]:
for i in range(3):
    base_name = f"weights/conv{i}_{{}}.npy"
    kernel, bias = model.layers[i].weights
    np.save(base_name.format("kernel"), kernel.numpy())
    np.save(base_name.format("bias"), bias.numpy())
    
kernel, bias = model.layers[4].weights
np.save("weights/dense_kernel.npy", kernel.numpy())
np.save("weights/dense_bias.npy", bias.numpy())